In [1]:
import os
import pandas as pd
import numpy as np
from collections import Counter
import pickle

import sys
sys.path.append(os.path.abspath('../../'))


with open('../../processed_data/selected_phenotypes.txt', 'rb') as reader:
    id2ph = [i.strip() for i in reader.readlines()]
    
with open('../src/ensembl2genename_GRCh38.85.p', 'rb') as reader:
    gid2gn = pickle.load(reader)

In [2]:
try:
    import cPickle as pickle
except:
    import pickle

In [3]:
folder_to_data = '../../data/'

In [4]:
#!/usr/bin/env python
import pandas
import sqlite3
from scipy.stats import norm 
import statsmodels.stats.multitest as multi

In [5]:
import gwas_component.gwas_component as gc

In [6]:
connection = sqlite3.connect(folder_to_data + "/gwas_g2p/metaxcan_results_p/metaxcan_results_v1.5.db")
res_all = {}

for phenotype in id2ph:
    res_all[phenotype] = []
    
    query = 'SELECT g.gene_name, m.zscore, m.n_snps_used, m.n_snps_model, p.tag as phenotype, t.tissue as tissue, g.gene ' 
    query += ' FROM gene AS g INNER JOIN metaxcan_result AS m ON g.id = m.gene_id' 
    query += ' INNER JOIN tissue AS t ON t.id = m.tissue_id  INNER JOIN pheno AS p ON p.id = m.pheno_id'
    query += ' WHERE p.tag = \'%s\'' % phenotype
    pi = pandas.read_sql_query(query, connection)
    pi = pi[pi.tissue != 'DGN_WB']

    id2tissue = sorted(set(pi.tissue))

    for tissue, item0 in pi.groupby('tissue'):
        tissue0 = tissue.replace('-', '_')
        
        gene2z = {g:z for g, z in zip(item0.gene, item0.zscore)}  
            
        ## gwas component score
        gc_model = gc.Coexpression_Model(tissue0)
        gc_model.compute_z(gene2z)
        
        drgenes = gc_model.get_driver_genes()
        res_all[phenotype] += drgenes
        
with open('./gcomp_drive_genes.p', 'wb') as writer:
    pickle.dump(res_all, writer)
    